In [2]:
import cv2
import numpy as np
import logging
import os
import subprocess
from collections import deque


def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "processing.log")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    logging.info(f"Logging configurato. File di log salvato in: {log_file}")


def temporal_smoothing_flow(
    video_path,
    output_dir,
    flow_threshold=0.5,
    alpha_fraction=0.2,
    window_size=30,
    morph_kernel=2,
    save_name="overlay.mp4",
    mask_save_name="mask.mp4",
    margin=10,
    scale_factor=0.2
):
    """
    Rileva il movimento nel video e genera:
    - Video con overlay che evidenzia le aree di movimento.
    - Video con maschera binaria che rappresenta le zone di movimento.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    overlay_path = os.path.join(output_dir, save_name)
    mask_path = os.path.join(output_dir, mask_save_name)

    out_overlay = cv2.VideoWriter(overlay_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height))
    out_mask = cv2.VideoWriter(mask_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height), isColor=False)

    ret, first_frame = cap.read()
    if not ret:
        logging.error("Impossibile leggere il primo frame del video.")
        return

    prev_gray_full = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.resize(prev_gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

    mask_queue = deque(maxlen=window_size)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_kernel, morph_kernel))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_full = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 2, 9, 2, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=False)
        mask_current_resized = (mag > flow_threshold).astype(np.uint8) * 255
        mask_original = cv2.resize(mask_current_resized, (width, height), interpolation=cv2.INTER_NEAREST)

        mask_queue.append(mask_original)
        cumulative_mask = np.sum(np.array(mask_queue), axis=0)
        mask_smoothed = (cumulative_mask >= (alpha_fraction * len(mask_queue) * 255)).astype(np.uint8) * 255
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_CLOSE, kernel)
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(mask_smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask_rect = np.zeros((height, width), dtype=np.uint8)
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x_padded = max(0, x - margin)
            y_padded = max(0, y - margin)
            w_padded = min(w + 2 * margin, width - x_padded)
            h_padded = min(h + 2 * margin, height - y_padded)
            cv2.rectangle(mask_rect, (x_padded, y_padded), (x_padded + w_padded, y_padded + h_padded), 255, -1)

        out_overlay.write(frame)
        out_mask.write(mask_rect)

        prev_gray = gray.copy()

    cap.release()
    out_overlay.release()
    out_mask.release()


def compress_with_ffmpeg(input_path, output_path):
    """
    Utilizza FFmpeg per applicare una compressione HEVC al video.
    """
    logging.info(f"Avvio della compressione HEVC con FFmpeg per: {input_path}")
    ffmpeg_command = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libx265", # Codec HEVC
        "-crf", "26",  # Livello di compressione (più alto = più compresso)
        "-preset", "fast",  # Ottimizza velocità e compressione
        "-pix_fmt", "yuv420p", # Formato pixel per la compatibilità
        "-y",
        output_path
    ]
    try:
        subprocess.run(ffmpeg_command, check=True)
        logging.info(f"Compressione completata. Video salvato in: {output_path}")
    except subprocess.CalledProcessError as e:
        logging.error(f"Errore durante la compressione con FFmpeg: {e}")


def compress_with_motion(input_video, mask_video, output_dir):
    """
    Applica la compressione nelle aree statiche basata sulla maschera di movimento.
    """
    cap_input = cv2.VideoCapture(input_video)
    cap_mask = cv2.VideoCapture(mask_video)

    if not cap_input.isOpened():
        logging.error(f"Impossibile aprire il video originale: {input_video}")
        return
    if not cap_mask.isOpened():
        logging.error(f"Impossibile aprire il video maschera: {mask_video}")
        return

    fps = cap_input.get(cv2.CAP_PROP_FPS)
    width = int(cap_input.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_input.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_video = os.path.join(output_dir, "compressed.mp4")
    out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height))

    QTY_aggressive = np.full((8, 8), 100, dtype=np.float32)

    while True:
        ret_in, frame_in = cap_input.read()
        ret_mask, frame_mask = cap_mask.read()
        if not ret_in or not ret_mask:
            break

        if len(frame_mask.shape) == 3:
            frame_mask = cv2.cvtColor(frame_mask, cv2.COLOR_BGR2GRAY)

        frame_ycrcb = cv2.cvtColor(frame_in, cv2.COLOR_BGR2YCR_CB)
        channels = cv2.split(frame_ycrcb)

        for i in range(0, frame_mask.shape[0], 8):
            for j in range(0, frame_mask.shape[1], 8):
                if frame_mask[i:i+8, j:j+8].mean() == 0:
                    for c in range(3):
                        block = channels[c][i:i+8, j:j+8]
                        if block.shape == (8, 8):
                            dct_block = cv2.dct(block.astype(np.float32) - 128)
                            quantized_block = np.round(dct_block / QTY_aggressive) * QTY_aggressive
                            idct_block = cv2.idct(quantized_block) + 128
                            channels[c][i:i+8, j:j+8] = np.clip(idct_block, 0, 255)

        frame_processed = cv2.cvtColor(cv2.merge(channels), cv2.COLOR_YCR_CB2BGR)
        out.write(frame_processed)

    cap_input.release()
    cap_mask.release()
    out.release()

    # Comprimere ulteriormente con FFmpeg
    optimized_output = os.path.join(output_dir, "compressed_final.mp4")
    compress_with_ffmpeg(output_video, optimized_output)


def main():
    input_dir = "../Dataset/input/"
    output_dir = "../Dataset/output/"
    input_video = [f for f in os.listdir(input_dir) if f.endswith(".mp4")][0]
    video_name = os.path.splitext(input_video)[0]
    video_output_dir = os.path.join(output_dir, video_name)

    if not os.path.exists(video_output_dir):
        os.makedirs(video_output_dir)

    setup_logging(video_output_dir)

    temporal_smoothing_flow(
        video_path=os.path.join(input_dir, input_video),
        output_dir=video_output_dir,
        save_name=f"{video_name}_overlay.mp4",
        mask_save_name=f"{video_name}_mask.mp4",
    )

    compress_with_motion(
        input_video=os.path.join(video_output_dir, f"{video_name}_overlay.mp4"),
        mask_video=os.path.join(video_output_dir, f"{video_name}_mask.mp4"),
        output_dir=video_output_dir
    )


if __name__ == "__main__":
    main()

2025-01-19 10:43:48,656 - INFO - Logging configurato. File di log salvato in: ../Dataset/output/test2/processing.log
2025-01-19 10:59:54,918 - INFO - Video compresso salvato in: ../Dataset/output/test2/compressed.mp4
